In [ ]:
!pip --quiet install https://github.com/scarfboy/wetsuite-dev/archive/refs/heads/main.zip

In [ ]:
import wetsuite.datasets
gem = wetsuite.datasets.load('gemeentes')
print( gem.description )

In [41]:
import pprint, random

pprint.pprint( random.choice( gem.data ) )  # show details for one random gemeente

{'Aantal inwoners': '25627',
 'Bevat plaatsen': ['Voorschoten'],
 'CBSCode': '0626',
 'Namen': ['Voorschoten', 'Gemeente Voorschoten'],
 'OWMS URI': 'http://standaarden.overheid.nl/owms/terms/Voorschoten_(gemeente)',
 'Oppervlakte': [11, 'km2'],
 'Organisatiecode': 'gm0626',
 'Overlaps with': [['Hoogheemraadschap van Rijnland'], ['Zuid-Holland']],
 'Predecessors': [],
 'Raad': [['VVD', 4],
          ['CDA', 4],
          ['GroenLinks', 4],
          ['Voorschoten Lokaal', 4],
          ['D66', 3],
          ['SP', 1],
          ['PvdA', 1]],
 'Service area of': [['1600',
                      'RDOG HM',
                      'Regionale dienst openbare gezondheidszorg Hollands '
                      'Midden',
                      'RDOG Hollands Midden'],
                     ['BSGR', 'Belastingsamenwerking Gouwe-Rijnland'],
                     ['Inkoopbureau H-10'],
                     ['Omgevingsdienst West-Holland', 'ODWH'],
                     ['Regiokorps Politie Hollands Midde

## Beleidsregels per gemeente

This uses the data together to do a specific search into the KOOP repositories (see also the datacollect_koop_repos example, for more introduction),
one per municipality.

Which uses nothing beyond the names.

In [2]:
import wetsuite.datacollect.koop_repositories
import wetsuite.helpers.etree as etree

# this uses only One reason for this not-really-a-dataset was the names, so we can do things like look for beleidsregels per gemeente

def show_result_title(record):
    ''' Take the title out of the search results. 
        Ignore how this code works for now, because we absolutel need to make this easier for you to do.
    '''
    gzd = record.find('recordData')[0]
    owmskern     = wetsuite.helpers.etree.kvelements_to_dict( originalData.find('meta/owmskern')   )
    owmsmantel   = wetsuite.helpers.etree.kvelements_to_dict( originalData.find('meta/owmsmantel') ) 
    cvdripm      = wetsuite.helpers.etree.kvelements_to_dict( originalData.find('meta/cvdripm')     ) 
    enrichedData = wetsuite.helpers.etree.kvelements_to_dict( gzd.find('enrichedData') )

    merged = {}
    merged.update(owmskern)
    merged.update(owmsmantel)
    merged.update(cvdripm)
    merged.update(enrichedData)


for gemeente_dict in gem.data[63:70]:  # the weird offset is tring to find Den Haag with its other name, to check that the search is not tripping over that
    query_gnames = ' OR '.join( '(creator any "%s")'%naam  for naam in gemeente_dict['Namen'] )

    # Complex-looking query to look for 
    #   (one of the gemeentenames)  AND (   'damocles'   OR   (something mentioning both drugs or the opiumwet  AND  one of the words you likely see around putting people out of their house )
    # This is a practical consideration: we _will_ get too many results, but at least what we want is probably in there 
    query = '(%s) AND ( (body any "damoclesbeleid damocles")  OR  (body any "drugs softdrugs harddrugs handelshoeveelheid opiumwet 13b") AND (body any "sluiting herstelsanctie bestuursdwang"))'%( 
        query_gnames
    )
    #print( query )

    cvdr = wetsuite.datacollect.koop_repositories.CVDR()
    cvdr.search_retrieve( query ) # search and fetch only first page, just so that num_records is filled in 
    print( "%3d  hits for   %s "%(cvdr.num_records(), ' / '.join(gemeente_dict['Namen'])) )

    cvdr.search_retrieve_many( query, callback=show_result_title ) # all results, and show titles

    #print()



ImportError: No module named wetsuite.datacollect.koop_repositories